In [1]:
import pandas as pd

In [2]:
# Load the datasets
ais_train = pd.read_csv('ais_train.csv', sep='|')
schedules = pd.read_csv('schedules_to_may_2024.csv', sep='|')

# Ensure datetime format
ais_train['time'] = pd.to_datetime(ais_train['time'], errors='coerce')
schedules['arrivalDate'] = pd.to_datetime(schedules['arrivalDate'], errors='coerce')
schedules['sailingDate'] = pd.to_datetime(schedules['sailingDate'], errors='coerce')

In [3]:
# Group ais_train by vesselId and sort by time
ais_grouped = ais_train.groupby('vesselId').apply(lambda x: x.sort_values('time')).reset_index(drop=True)

/tmp/ipykernel_2488/1036389009.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ais_grouped = ais_train.groupby('vesselId').apply(lambda x: x.sort_values('time')).reset_index(drop=True)


In [4]:
# Initialize a DataFrame to hold results
results = []

for vessel_id in schedules['vesselId'].unique():
    # Get the schedule for the current vessel
    vessel_schedule = schedules[schedules['vesselId'] == vessel_id].reset_index(drop=True)
    
    # Get the corresponding ais_train entries for the current vessel
    vessel_data = ais_grouped[ais_grouped['vesselId'] == vessel_id]
    
    current_schedule_index = 0
    
    # Iterate through the ais_train data for this vessel
    for idx, row in vessel_data.iterrows():
        # Check if there are any ports left in the schedule
        if current_schedule_index < len(vessel_schedule):
            expected_port_id = vessel_schedule.loc[current_schedule_index, 'portId']
            
            # Check if the current portId matches the expected portId
            if row['portId'] == expected_port_id:
                # Port matches, move to the next expected port
                current_schedule_index += 1
                results.append({
                    'vesselId': vessel_id,
                    'current_portId': row['portId'],
                    'expected_portId': expected_port_id,
                    'status': 'Reached',
                    'row_number': row.name  # Capture the original index from ais_train
                })
            else:
                # Port does not match
                results.append({
                    'vesselId': vessel_id,
                    'current_portId': row['portId'],
                    'expected_portId': expected_port_id,
                    'status': 'Mismatch',
                    'row_number': row.name  # Capture the original index from ais_train
                })
        else:
            # No more ports left in the schedule
            break

In [24]:
# Filter rows based on 'portId' and 'vesselId' conditions
filtered_df = schedules[(schedules['portId'] == '61d375e793c6feb83e5eb3e2') & (schedules['vesselId'] == '61e9f3b1b937134a3c4bfe53')].sort_values(by='arrivalDate', ascending=True)

pd.set_option('display.max_rows', None)

# Display the filtered rows
print(filtered_df)

                        vesselId            shippingLineId            shippingLineName                arrivalDate                sailingDate             portName                    portId  portLatitude  portLongitude
102764  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean  2023-01-03 00:00:00+00:00  2023-01-07 00:00:00+00:00  Port of Bremerhaven  61d375e793c6feb83e5eb3e2     53.563611       8.554722
46721   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean  2023-01-03 00:00:00+00:00  2023-01-07 00:00:00+00:00  Port of Bremerhaven  61d375e793c6feb83e5eb3e2     53.563611       8.554722
69098   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean  2023-06-25 00:00:00+00:00  2023-06-26 00:00:00+00:00  Port of Bremerhaven  61d375e793c6feb83e5eb3e2     53.563611       8.554722
69102   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean  2023-06-25 00:00:00+00:00  2023-06-26 00:00:

In [5]:
# Convert results to a DataFrame
results_df = pd.DataFrame(results)

pd.set_option('display.width', 20000) 

# Print results
print(results_df)

                        vesselId            current_portId           expected_portId    status  row_number
0       61e9f3b1b937134a3c4bfe53  61d36f640a1807568ff9a103  61d38499b7b7526e1adf3d54  Mismatch      163498
1       61e9f3b1b937134a3c4bfe53  61d36f640a1807568ff9a103  61d38499b7b7526e1adf3d54  Mismatch      163499
2       61e9f3b1b937134a3c4bfe53  61d36f640a1807568ff9a103  61d38499b7b7526e1adf3d54  Mismatch      163500
3       61e9f3b1b937134a3c4bfe53  61d36f640a1807568ff9a103  61d38499b7b7526e1adf3d54  Mismatch      163501
4       61e9f3b1b937134a3c4bfe53  61d36f640a1807568ff9a103  61d38499b7b7526e1adf3d54  Mismatch      163502
...                          ...                       ...                       ...       ...         ...
488270  61e9f43ab937134a3c4c0151  634c4de270937fc01c3a7015  61d37a581366c3998241d985  Mismatch      868161
488271  61e9f43ab937134a3c4c0151  634c4de270937fc01c3a7015  61d37a581366c3998241d985  Mismatch      868162
488272  61e9f43ab937134a3c4c0151  634

In [6]:
# Load the schedules dataset
schedules = pd.read_csv('schedules_to_may_2024.csv', sep='|')

# Filter rows where vesselId is 61e9f3a8b937134a3c4bfdf7
filtered_rows = schedules[schedules['vesselId'] == '61e9f3b1b937134a3c4bfe53']

# Ensure 'arrivalDate' is in datetime format
filtered_rows['arrivalDate'] = pd.to_datetime(filtered_rows['arrivalDate'], utc=True)

# Sort the filtered rows by 'arrivalDate'
sorted_rows = filtered_rows.sort_values(by='arrivalDate')

# Define the date range for filtering in UTC
start_date = pd.to_datetime('2024-01-01', utc=True)
end_date = pd.to_datetime('2024-05-07', utc=True)

# Filter out rows outside the date range
filtered_sorted_rows = sorted_rows[(sorted_rows['arrivalDate'] >= start_date) & (sorted_rows['arrivalDate'] <= end_date)]

# Print the filtered and sorted rows
print(filtered_sorted_rows)

                        vesselId            shippingLineId            shippingLineName               arrivalDate                sailingDate             portName                    portId  portLatitude  portLongitude
131391  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-01 00:00:00+00:00  2024-01-05 00:00:00+00:00  Port of Port Kembla  61d36f770a1807568ff9a126    -34.462500     150.899444
47836   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-01 00:00:00+00:00  2024-01-05 00:00:00+00:00  Port of Port Kembla  61d36f770a1807568ff9a126    -34.462500     150.899444
135245  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-01 00:00:00+00:00  2024-01-03 00:00:00+00:00  Port of Port Kembla  61d36f770a1807568ff9a126    -34.462500     150.899444
44004   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-02 00:00:00+00:00  2024-01-15 00:00:00+00

/tmp/ipykernel_2488/2838089976.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['arrivalDate'] = pd.to_datetime(filtered_rows['arrivalDate'], utc=True)


In [7]:
print(filtered_sorted_rows.drop_duplicates())

                        vesselId            shippingLineId            shippingLineName               arrivalDate                sailingDate                           portName                    portId  portLatitude  portLongitude
131391  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-01 00:00:00+00:00  2024-01-05 00:00:00+00:00                Port of Port Kembla  61d36f770a1807568ff9a126    -34.462500     150.899444
135245  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-01 00:00:00+00:00  2024-01-03 00:00:00+00:00                Port of Port Kembla  61d36f770a1807568ff9a126    -34.462500     150.899444
44004   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-02 00:00:00+00:00  2024-01-15 00:00:00+00:00                  Port of Melbourne  61d36f5f0a1807568ff9a0fa    -37.832778     144.917500
50392   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen

In [8]:
unique_port_df2 = filtered_sorted_rows.drop_duplicates(subset='portId')

# Display the DataFrame with unique 'portId' values
print(unique_port_df2)

                        vesselId            shippingLineId            shippingLineName               arrivalDate                sailingDate                           portName                    portId  portLatitude  portLongitude
131391  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-01 00:00:00+00:00  2024-01-05 00:00:00+00:00                Port of Port Kembla  61d36f770a1807568ff9a126    -34.462500     150.899444
44004   61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-02 00:00:00+00:00  2024-01-15 00:00:00+00:00                  Port of Melbourne  61d36f5f0a1807568ff9a0fa    -37.832778     144.917500
103179  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen Ocean 2024-01-10 00:00:00+00:00  2024-01-15 00:00:00+00:00                     Fremantle Port  61d36f5d0a1807568ff9a0f7    -32.047500     115.740833
103948  61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac  Wallenius Wilhelmsen

In [9]:
# Get distinct vesselId's from the schedules dataset
distinct_vessel_ids = schedules['vesselId'].unique()

# Convert to a DataFrame for better visualization
distinct_vessel_ids_df = pd.DataFrame(distinct_vessel_ids, columns=['vesselId'])

# Display the distinct vessel IDs
print(distinct_vessel_ids_df)

                     vesselId
0    61e9f3b1b937134a3c4bfe53
1    61e9f3acb937134a3c4bfe25
2    61e9f3fdb937134a3c4bffe1
3    61e9f3a2b937134a3c4bfdd7
4    61e9f3a8b937134a3c4bfdfb
..                        ...
247  61e9f3fdb937134a3c4bffdf
248  61e9f401b937134a3c4c0001
249  61e9f424b937134a3c4c00cd
250  61e9f435b937134a3c4c0129
251  61e9f43ab937134a3c4c0151

[252 rows x 1 columns]


In [10]:
ais_test = pd.read_csv('ais_test.csv')

# Get distinct vesselId's from ais_test and ais_train
distinct_vessel_ids_test = ais_test['vesselId'].unique()
distinct_vessel_ids_train = ais_train['vesselId'].unique()

# Find missing vesselId's in ais_test and ais_train
missing_in_test = set(distinct_vessel_ids) - set(distinct_vessel_ids_test)
missing_in_train = set(distinct_vessel_ids) - set(distinct_vessel_ids_train)

# Print the missing vesselIds for each dataframe
print("Missing vesselId's in ais_test:")
print(len(missing_in_test))
print(missing_in_test)

print("\nMissing vesselId's in ais_train:")
print(len(missing_in_train))
print(missing_in_train)

Missing vesselId's in ais_test:
185
{'6207e03d66fc0a8e43c61223', '61e9f3a7b937134a3c4bfdeb', '61e9f3feb937134a3c4bffe7', '61e9f44fb937134a3c4c01d7', '61e9f3a3b937134a3c4bfddb', '61e9f3f4b937134a3c4bffb1', '61e9f3b7b937134a3c4bfe83', '61e9f41cb937134a3c4c0089', '61e9f396b937134a3c4bfda9', '61e9f430b937134a3c4c010d', '61e9f3a8b937134a3c4bfdf5', '61e9f3f9b937134a3c4bffcd', '61e9f3b6b937134a3c4bfe7d', '61e9f3f1b937134a3c4bff91', '61e9f3f4b937134a3c4bffaf', '61e9f3ffb937134a3c4bfff1', '61e9f417b937134a3c4c0071', '61e9f400b937134a3c4bfff5', '61e9f456b937134a3c4c01ff', '61e9f439b937134a3c4c014f', '61e9f42eb937134a3c4c0105', '61e9f3f2b937134a3c4bff9f', '61e9f41eb937134a3c4c00a1', '61e9f39bb937134a3c4bfdbd', '61e9f47bb937134a3c4c02fd', '61e9f431b937134a3c4c0113', '61e9f3abb937134a3c4bfe1f', '61e9f3a5b937134a3c4bfde1', '61e9f44ab937134a3c4c01bb', '61e9f3adb937134a3c4bfe31', '61e9f439b937134a3c4c014d', '61e9f3f5b937134a3c4bffbb', '61e9f3a8b937134a3c4bfdfb', '61e9f3aeb937134a3c4bfe45', '61e9f3f2b9

In [11]:
# Get distinct vesselId's from schedules
distinct_vessel_ids_schedules = schedules['vesselId'].unique()

# Find missing vesselId's in schedules from ais_test and ais_train
missing_in_schedules_from_test = set(distinct_vessel_ids_test) - set(distinct_vessel_ids_schedules)
missing_in_schedules_from_train = set(distinct_vessel_ids_train) - set(distinct_vessel_ids_schedules)

# Count of missing vesselId's
missing_count_test = len(missing_in_schedules_from_test)
missing_count_train = len(missing_in_schedules_from_train)

# Print results
print(f"Number of vesselId's in ais_test: {len(distinct_vessel_ids_test)} of which we are missing {missing_count_test} in schedules")
print(f"Number of vesselId's in ais_train: {len(distinct_vessel_ids_train)} of which we are missing {missing_count_train} in schedules")

Number of vesselId's in ais_test: 215 of which we are missing 148 in schedules
Number of vesselId's in ais_train: 688 of which we are missing 449 in schedules


In [12]:
# Extract the unique vesselIds
test_vessel_ids = set(ais_test['vesselId'].unique())
train_vessel_ids = set(ais_train['vesselId'].unique())

# Find vesselIds in ais_test but not in ais_train
missing_vessel_ids = test_vessel_ids - train_vessel_ids

# Count and print the number of missing vesselIds
print(f"Number of vesselIds in ais_test but not in ais_train: {len(missing_vessel_ids)}")

Number of vesselIds in ais_test but not in ais_train: 0


In [13]:
print(len(ais_train))

1522065


In [14]:
import numpy as np
from geopy.distance import great_circle

In [18]:
# Ensure 'arrivalDate' is in datetime format
filtered_sorted_rows['arrivalDate'] = pd.to_datetime(filtered_sorted_rows['arrivalDate'], utc=True)

# Function to calculate distance between two points (latitude, longitude)
def calculate_distance(lat1, lon1, lat2, lon2):
    return great_circle((lat1, lon1), (lat2, lon2)).kilometers

# Initialize a list to store results
arrival_info = []

# Iterate over each vesselId in the schedules
for vessel_id in filtered_sorted_rows['vesselId'].unique():
    # Filter the schedule for the current vessel
    vessel_schedule = filtered_sorted_rows[schedules['vesselId'] == vessel_id]

    # Get the vessel's position history from ais_train
    vessel_positions = ais_train[ais_train['vesselId'] == vessel_id]

    # Iterate through each port in the vessel's schedule
    for _, port_row in vessel_schedule.iterrows():
        expected_arrival = port_row['arrivalDate']
        port_lat = port_row['portLatitude']
        port_lon = port_row['portLongitude']
        port_name = port_row['portName']

        # Check when the vessel arrives at this port
        for _, position_row in vessel_positions.iterrows():
            vessel_lat = position_row['latitude']
            vessel_lon = position_row['longitude']
            time = position_row['time']  # Ensure this column exists in ais_train

            # Calculate distance to the port
            distance_to_port = calculate_distance(vessel_lat, vessel_lon, port_lat, port_lon)

            # Check if the vessel is close enough to the port
            if distance_to_port < 6:  # Vessel is considered arrived
                arrival_info.append({
                    'vesselId': vessel_id,
                    'expectedArrival': expected_arrival,
                    'actualArrival': time,
                    'portLatitude': port_lat,
                    'portLongitude': port_lon,
                    'portName': port_name
                })
                break  # Exit the loop once the vessel has arrived

# Convert results to a DataFrame for further analysis
arrival_df = pd.DataFrame(arrival_info)

# Print the resulting DataFrame
print(arrival_df)

/tmp/ipykernel_2488/127239161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_sorted_rows['arrivalDate'] = pd.to_datetime(filtered_sorted_rows['arrivalDate'], utc=True)
/tmp/ipykernel_2488/127239161.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vessel_schedule = filtered_sorted_rows[schedules['vesselId'] == vessel_id]


                     vesselId           expectedArrival       actualArrival  portLatitude  portLongitude                           portName
0    61e9f3b1b937134a3c4bfe53 2024-01-02 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
1    61e9f3b1b937134a3c4bfe53 2024-01-02 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
2    61e9f3b1b937134a3c4bfe53 2024-01-02 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
3    61e9f3b1b937134a3c4bfe53 2024-01-02 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
4    61e9f3b1b937134a3c4bfe53 2024-01-02 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
..                        ...                       ...                 ...           ...            ...                                ...
217  61e9f3b1b937134

In [19]:
# Assuming your data is in a DataFrame called df
unique_rows = arrival_df.drop_duplicates().sort_values(by='actualArrival', ascending=True)

# Display the unique rows
print(unique_rows)

                     vesselId           expectedArrival       actualArrival  portLatitude  portLongitude                           portName
212  61e9f3b1b937134a3c4bfe53 2024-04-29 00:00:00+00:00 2024-01-01 00:15:01    -27.382500     153.169444                   Port of Brisbane
0    61e9f3b1b937134a3c4bfe53 2024-01-02 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
5    61e9f3b1b937134a3c4bfe53 2024-01-04 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
7    61e9f3b1b937134a3c4bfe53 2024-01-06 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
9    61e9f3b1b937134a3c4bfe53 2024-01-07 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
12   61e9f3b1b937134a3c4bfe53 2024-01-08 00:00:00+00:00 2024-01-12 15:54:58    -37.832778     144.917500                  Port of Melbourne
18   61e9f3b1b937134